In [ ]:
%load_ext autoreload
%autoreload 2

import re
from typing import Tuple
from pathlib import Path
from datetime import timedelta

import pandas as pd
file_path = Path("_chat.txt")

In [ ]:
from whatsapp_parser import extract_messages, cleanup, WhatsAppGroupAnalysis

df = extract_messages(file_path=file_path)
print(f"Before cleanup: {len(df)}")
df = cleanup(df)
print(f"After cleanup: {len(df)}")
df

In [ ]:
# Parse datetime
# df['Datetime'] = pd.to_datetime(df['Datetime'])
# df

In [ ]:
# Find latest date from df
latest_date = df['Datetime'].max().strftime("%Y%m%d")
file_name = f"../{latest_date}_Messages.csv"

# Save the dataframe as a csv file with {latest_date}_Messages.csv as the filename.
df.to_csv(file_name, index=False, encoding='utf-8', header=True, sep="|")
pd.read_csv(file_name, sep="|")

In [ ]:
# Create an instance of the class using the uploaded CSV file
analysis = WhatsAppGroupAnalysis(df)

# Test the methods
current_users_df = analysis.get_current_users()
message_count_in_window_df = analysis.get_message_count_in_window(60)

In [ ]:
inactive_users_to_remove = analysis.get_inactive_users()

In [ ]:
# Sort by total messages sent and then by joining date
inactive_users = inactive_users_to_remove.sort_values(
    by=['Total_Messages_Sent', 'Joining_Date'], ascending=[True, True])
# Display the DataFrame
inactive_users

In [ ]:
inactive_users.to_csv('inactive_users.csv', index=False)